# PyTorch Cifar10 local training  

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
!/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [9]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-fastmoe'

role = sagemaker.get_execution_role()

In [3]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local_gpu


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [4]:
# data_inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/OpenNRE/benchmark/finre', bucket=bucket, key_prefix='data/finre')
# pretrain_inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/OpenNRE/pretrain/baidubaike', bucket=bucket, key_prefix='pretrain/baidubaike')
data_inputs = 'file:///home/ec2-user/SageMaker/OpenNRE/benchmark/finre'
pretrain_inputs = 'file:///home/ec2-user/SageMaker/OpenNRE/pretrain/baidubaike'
inputs = {'training': data_inputs.replace('/benchmark/finre', ''), 'pretrain': pretrain_inputs}
print(inputs)

{'training': 'file:///home/ec2-user/SageMaker/OpenNRE', 'pretrain': 'file:///home/ec2-user/SageMaker/OpenNRE/pretrain/baidubaike'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [5]:
from sagemaker.pytorch import PyTorch

# git_config = {'repo': 'https://github.com/whn09/OpenNRE.git', 'branch': 'master'}
hyperparameters = {'max_epoch': 2}

estimator = PyTorch(entry_point='train_test_finre_supervised_bert.py',
                            source_dir='.',
                            #git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.6.0',
                            py_version='py3',
                            script_mode=True,
                            train_instance_count=1,
                            train_instance_type=instance_type)

estimator.fit(inputs)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Building with native build. Learn about native build in Compose here: https://docs.docker.com/go/compose-native-build/
Creating lh84xoedo7-algo-1-o7s25 ... 
Creating lh84xoedo7-algo-1-o7s25 ... done
Attaching to lh84xoedo7-algo-1-o7s25
lh84xoedo7-algo-1-o7s25 | 2021-02-24 01:40:12,811 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
lh84xoedo7-algo-1-o7s25 | 2021-02-24 01:40:12,836 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
lh84xoedo7-algo-1-o7s25 | 2021-02-24 01:40:12,840 sagemaker_pytorch_container.training INFO     Invoking user training script.
lh84xoedo7-algo-1-o7s25 | 2021-02-24 01:40:26,906 sagemaker-training-toolkit INFO     Installing module with the following command:
lh84xoedo7-algo-1-o7s25 | /opt/conda/bin/python -m pip install . -r requirements.txt
lh84xoedo7-algo-1-o7s25 | Processing /opt/ml/code
lh84xoedo7-algo-1-o7s25 | Requirement already satisfied: torch==1.6.0 in /opt/conda/lib/py

In [7]:
data_inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/OpenNRE/benchmark/finre', bucket=bucket, key_prefix='training/benchmark/finre')
pretrain_inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/OpenNRE/pretrain/baidubaike', bucket=bucket, key_prefix='pretrain/baidubaike')
inputs = {'training': data_inputs.replace('/benchmark/finre', ''), 'pretrain': pretrain_inputs}
print(inputs)

{'training': 's3://sagemaker-us-east-1-579019700964/training', 'pretrain': 's3://sagemaker-us-east-1-579019700964/pretrain/baidubaike'}


In [10]:
from sagemaker.pytorch import PyTorch

hyperparameters = {'max_epoch': 2}

instance_type = 'ml.p3.2xlarge'

estimator = PyTorch(entry_point='train_test_finre_supervised_bert.py',
                            source_dir='.',
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.6.0',
                            py_version='py3',
                            script_mode=True,
                            train_instance_count=1,
                            train_instance_type=instance_type)

estimator.fit(inputs)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-02-24 04:59:49 Starting - Starting the training job...
2021-02-24 05:00:10 Starting - Launching requested ML instancesProfilerReport-1614142727: InProgress
......
2021-02-24 05:01:14 Starting - Preparing the instances for training.........
2021-02-24 05:02:33 Downloading - Downloading input data...
2021-02-24 05:03:13 Training - Downloading the training image......
2021-02-24 05:04:16 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-24 05:04:16,867 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-02-24 05:04:16,891 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-02-24 05:04:16,899 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-02-24 05:04:32,042 sagemaker-training-toolkit INFO     Installing module with the following c

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

# from sagemaker.pytorch.model import PyTorchModel

# pytorch_model = PyTorchModel(model_data='s3://{}/sagemaker/DEMO-pytorch-mnist/model.tar.gz'.format(bucket), role=role,
#                              entry_point='train_test_finre_supervised_bert.py', framework_version='1.6.0', py_version='py3')

# predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

# Invoking the endpoint

In [ ]:
# from sagemaker.predictor import json_serializer, json_deserializer

# predictor.accept = 'application/json'
# predictor.content_type = 'application/json'

# predictor.serializer = json_serializer
# predictor.deserializer = json_deserializer

# data = {"token": ["<N>", "日", ",", "“", "借", "壳", "”", "华", "盛", "达", "的", "刚", "泰", "控", "股", "在", "上", "交", "所", "举", "行", "了", "更", "名", "及", "上", "市", "仪", "式", "。"], "h": {"name": "华盛达", "id": "", "pos": [7, 10]}, "t": {"name": "刚泰控股", "id": "", "pos": [11, 15]}, "relation": "拥有"}

# outputs = predictor.predict(data)

# print('outputs: ', outputs)

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()